In [1]:
import numpy as np
import robosuite as suite

In [2]:
from robosuite.models import MujocoWorldBase

world = MujocoWorldBase()

In [3]:
from robosuite.models.robots import IIWA

mujoco_robot = IIWA()

In [4]:
from robosuite.models.grippers import gripper_factory

gripper = gripper_factory('Robotiq140Gripper')
mujoco_robot.add_gripper(gripper)

In [5]:
mujoco_robot.set_base_xpos([0, 0, 0.8])
world.merge(mujoco_robot)

In [6]:
from robosuite.models.arenas import TableArena

mujoco_arena = TableArena()
mujoco_arena.set_origin([0.8, 0, 0])
world.merge(mujoco_arena)

In [7]:
from robosuite.models.objects import ClothObject
from robosuite.utils.mjcf_utils import new_joint

test = ClothObject(name="test")
test_obj = test.get_obj()
test_obj.set('pos', '1.0 0 2.0')

world.worldbody.append(test_obj)
# world.merge(test)
# from robosuite.models.objects import DoorObject
# from robosuite.utils.mjcf_utils import new_joint

# test = DoorObject(name="test")
# test_obj = test.get_obj()
# test_obj.set('pos', '1.0 0 1.0')

# world.merge(test)

In [8]:
model = world.get_model(mode="mujoco_py")

In [9]:
from mujoco_py import MjSim, MjViewer

sim = MjSim(model)
viewer = MjViewer(sim)
viewer.vopt.geomgroup[0] = 0 # disable visualization of collision mesh

for i in range(10000):
  sim.data.ctrl[:] = 0
  sim.step()
  viewer.render()

Creating window glfw


SystemExit: 0

/home/yiliu77/Desktop/Python/envs/main_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3435: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
